In [173]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as pl

def target(wsize, alphabet):
    return np.random.choice(alphabet, wsize)

def round_up(num):
    return (10 - num % 10) + num

class Chromosome:

    genes = []
    fitness = 0

    def __init__(self, size, alphabet):
        np.random.seed()
        self.genes = np.random.choice(alphabet, size) 

    def __str__(self):
        return "{0}".format(self.genes)

    def _fitness(self, target):
        hits = 0
        for i, j in zip(self.genes, target):
            if i == j:
                hits += 1
        self.fitness = (hits*100)/len(target)

class Population:

    def __init__(self, psize, csize, alphabet):
        self.alphabet = alphabet
        self.psize = psize
        self.csize = csize
        self.population = self._gen_pop(psize, csize, alphabet)
        
    def __str__(self):
        return "\n".join(map(str,self.population))

    def sort(self):
        self.population = sorted(self.population, key=lambda chromosome: chromosome.fitness)

    def renew(self):
        index = int(len(self.population)/2)
        self.population = self._gen_pop(index, self.csize, self.alphabet) \
        + self.population[index:]
    
    def _gen_pop(self, psize, csize, alphabet):
        pop = []
        for _ in range(psize):
            pop.append(Chromosome(csize, self.alphabet))
        return pop

class Randomwalk(Population):
    
    def __init__(self, psize, alphabet, target, gcounter, tcounter = 5):
        self.alphabet = alphabet
        self.target = target
        self.psize = psize
        self.csize = len(self.target)
        self.gcounter = gcounter
        self.tcounter = tcounter 

    def update(self, target):
        for i in range(len(self.population)):
            self.population[i]._fitness(self.target)    

    def get_fitness(self):
        return [self.population[i].fitness for i in range(len(self.population))]

    def run(self):
       
        self.best_scores = []
        
        print("{!s} started...".format(type(self).__name__))
        print("Target: {!s}".format(self.target))
        print("Population size: {!s}".format(self.psize))
        print("Total executions: {!s}".format(self.tcounter))
        print("Max generations without change: {!s}\n".format(self.gcounter))
        
        for i in range(self.tcounter):
            
            self.population = self._gen_pop(self.psize, self.csize, self.alphabet)
            self.update(self.target)
            
            self.scores = []
            
            #print("{0!s}a Execução:".format(i+1))
            j = 0
            k = 0
            
            while j < self.gcounter:
                
                self.sort()
                self.renew()
                self.update(self.target)
                self.sort()
                
                self.scores.append(self.get_fitness()[-1])
                
                if k > 10:
                    if self.scores[-1] == self.scores[-2]:
                        j += 1
                    else:
                        j = 0
                
                k += 1
                
                #print("{0!s}a Geração:\nMelhor resultado: {1!s}".format(j+1, self.scores[-1]))
                           
            self.best_scores.append(self.scores)
            
            print("Execution {0!s} ended in {1!s} generations".format(i+1, k))
            print("Target: {!s}".format(self.target))
            print("Result: {!s}".format(self.population[-1]))
            print("Best score: {!s}%\n".format(self.scores[-1]))
            
        """    
        self.bs = np.array(self.best_scores)
        self.m = np.mean(self.bs, axis=0) 
        self.std = np.std(self.bs, axis=0)
        self.idx = np.arange(0, self.gcounter)
        """
            
    def plot(self):
        
        pl.figure(figsize=(15, 10))
        pl.title(u"Média de Acertos por Geração")
        pl.xlabel(u"Gerações")
        pl.ylabel(u"Acerto(%)")
        pl.grid(alpha=0.3)
        pl.plot(self.m, color='black')
        pl.errorbar(self.idx, self.m, self.std, ls=None, alpha=0.5)
        pl.show()

In [174]:
t1 = target(20, [0, 1])
t2 = target(30, [0, 1])
t3 = target(40, [0, 1])

psize1  = 20
psize2  = 100
psize3  = 500
psize4  = 1000

gcount = 100

In [176]:
r11 = Randomwalk(psize=psize4, alphabet=[0,1], target=t3, gcounter=gcount, tcounter=10)

In [177]:
r11.run()
#r11.plot()

Randomwalk started...
Target: [1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0
 1 1 1]
Population size: 1000
Total executions: 10
Max generations without change: 100

Execution 1 ended in 166 generations
Target: [1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0
 1 1 1]
Result: [1 1 0 0 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 1 1 1]
Best score: 82.5%

Execution 2 ended in 207 generations
Target: [1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0
 1 1 1]
Result: [1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0
 1 1 1]
Best score: 90.0%

Execution 3 ended in 113 generations
Target: [1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0
 1 1 1]
Result: [1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1
 1 1 1]
Best score: 80.0%

Execution 4 ended in 172 generations
Target: [1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0

In [166]:
for i in range(len(r11.best_scores)):
    print(len(r11.best_scores[i]))

print()
print(max([len(l) for l in r11.best_scores]))



147
112
111
111
170
134
186
122
179
151

186


In [172]:
round_up(186)

190